# Introduction
Uses one ETF.

Model Inputs are indicators.

Multi models with dynamic training

# Results

In [1]:
import pandas as pd
from collections import Counter
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import datetime
import backtrader as bt
import sys
sys.path.insert(0, '../code')
import prepareDataV3 as util
import utils
import time
import math
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as pltNonConflict
import re
np.set_printoptions(precision=4)
import itertools

In [2]:
THRESHOLD = 0.00
TEST_PART = 0.2
IMAGE_PATH = '../output/'
IMAGE_SUFIX  = '.png'
RISK_TO_TRADE  = 0.01 # 0.01 = 1%

NUMBER_OF_RUNS = 10
DEBUG_VERBOSE = False

TRAINING_AMOUNT = 1500 # number used at previous notebooks
BIGGEST_INDICATOR_START_LOCKUP = 59
BACKTEST_START_LOCKUP = TRAINING_AMOUNT + BIGGEST_INDICATOR_START_LOCKUP
TRAINING_EXPIRE_IN = 60 # random number that goes till end of file

TESTS = [{
	'clf': MLPClassifier(activation='logistic', hidden_layer_sizes=(20, 20, 20)),
	'key': "logistic_(20, 20, 20)",
	'dataFileName': 'EURUSDDaily'
},{
	'clf': MLPClassifier(activation='relu', hidden_layer_sizes=(100,)),
	'key': "relu_(100,)",
	'dataFileName': 'EURUSDDaily'
},{
	'clf': MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 100, 100)),
	'key': "logistic_(100, 100, 100)",
	'dataFileName': 'EURUSDDaily'
},{
	'clf': MLPClassifier(activation='tanh', hidden_layer_sizes=(20, 20, 20, 20)),
	'key': "tanh_(20, 20, 20, 20)",
	'dataFileName': 'EURUSDDaily'
},{
	'clf': MLPClassifier(activation='tanh', hidden_layer_sizes=(100, 100, 100)),
	'key': "tanh_(100, 100, 100)",
	'dataFileName': 'EURUSDDaily'
},{
	'clf': MLPClassifier(activation='relu', hidden_layer_sizes=(20, 20, 20, 20)),
	'key': "relu_(20, 20, 20, 20)",
	'dataFileName': 'EURUSDDaily'
},{
	'clf': MLPClassifier(activation='tanh', hidden_layer_sizes=(20, 20)),
	'key': "tanh_(20, 20)",
	'dataFileName': 'EURUSDDaily'
},{
	'clf': MLPClassifier(activation='tanh', hidden_layer_sizes=(20, 20, 20)),
	'key': "tanh_(20, 20, 20)",
	'dataFileName': 'EURUSDDaily'
},{
	'clf': MLPClassifier(activation='relu', hidden_layer_sizes=(100, 100, 100)),
	'key': "relu_(100, 100, 100)",
	'dataFileName': 'EURUSDDaily'
},{
	'clf': MLPClassifier(activation='relu', hidden_layer_sizes=(20, 20, 20)),
	'key': "relu_(20, 20, 20)",
	'dataFileName': 'EURUSDDaily'
}]

DEFAULT_DT_FORMAT = '%Y.%m.%d'
DEFAULT_FROM_DATE = datetime.date(2010, 7, 1)
PLOT_FROM_DATE = datetime.datetime(2016, 6, 1, 0, 0, 0)

# ENUM
UP = 1
DOWN = 0

# Backtesting Strategy
Model SVM Linear (https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC)

In [3]:
class TestStrategy(bt.Strategy):
    def __init__(self):
        self.indicators = []
        self.toMean = {
            "trainScores": {"data": []},
            "predictions": []
        }

        # just to set minimum_period and skip first instance
        self.sma = bt.indicators.SimpleMovingAverage(period=BACKTEST_START_LOCKUP, plot=False)

        # risk for the sizer
        self.StandardDeviation = bt.indicators.StandardDeviation(plot=False)

        self.indicators.append(bt.indicators.AccelerationDecelerationOscillator(plot=False))
        self.indicators.append(bt.indicators.Accum(plot=False))
        self.indicators.append(bt.indicators.AdaptiveMovingAverage(plot=False))
        self.indicators.append(bt.indicators.AdaptiveMovingAverageEnvelope(plot=False))
        self.indicators.append(bt.indicators.AdaptiveMovingAverageOscillator(plot=False))
        self.indicators.append(bt.indicators.AllN(plot=False))
        self.indicators.append(bt.indicators.AnyN(plot=False))
        # self.indicators.append(bt.indicators.ApplyN(plot=False))
        self.indicators.append(bt.indicators.AroonDown(plot=False))
        self.indicators.append(bt.indicators.AroonOscillator(plot=False))
        self.indicators.append(bt.indicators.AroonUp(plot=False))
        self.indicators.append(bt.indicators.AroonUpDown(plot=False))
        self.indicators.append(bt.indicators.AroonUpDownOscillator(plot=False))
        self.indicators.append(bt.indicators.Average(plot=False))
        self.indicators.append(bt.indicators.AverageDirectionalMovementIndex(plot=False))
        self.indicators.append(bt.indicators.AverageDirectionalMovementIndexRating(plot=False))
        self.indicators.append(bt.indicators.AverageTrueRange(plot=False))
        self.indicators.append(bt.indicators.AwesomeOscillator(plot=False))
        # self.indicators.append(bt.indicators.BaseApplyN(plot=False))
        self.indicators.append(bt.indicators.BollingerBands(plot=False))
        self.indicators.append(bt.indicators.BollingerBandsPct(plot=False))
        # self.indicators.append(bt.indicators.CointN(plot=False))
        self.indicators.append(bt.indicators.CommodityChannelIndex(plot=False))
        # self.indicators.append(bt.indicators.CrossDown(plot=False))
        # self.indicators.append(bt.indicators.CrossOver(plot=False))
        # self.indicators.append(bt.indicators.CrossUp(plot=False))
        #self.indicators.append(bt.indicators.DV2(plot=False))                                    CHECK THIS ONE, SEEMS INTERESTING https://www.backtrader.com/docu/indautoref/#dv2
        self.indicators.append(bt.indicators.DemarkPivotPoint(plot=False))
        self.indicators.append(bt.indicators.DetrendedPriceOscillator(plot=False))
        self.indicators.append(bt.indicators.DicksonMovingAverage(plot=False))
        self.indicators.append(bt.indicators.DicksonMovingAverageEnvelope(plot=False))
        self.indicators.append(bt.indicators.DicksonMovingAverageOscillator(plot=False))
        self.indicators.append(bt.indicators.DirectionalIndicator(plot=False))
        self.indicators.append(bt.indicators.DirectionalMovement(plot=False))
        self.indicators.append(bt.indicators.DirectionalMovementIndex(plot=False))
        self.indicators.append(bt.indicators.DoubleExponentialMovingAverage(plot=False))
        self.indicators.append(bt.indicators.DoubleExponentialMovingAverageEnvelope(plot=False))
        self.indicators.append(bt.indicators.DoubleExponentialMovingAverageOscillator(plot=False))
        self.indicators.append(bt.indicators.DownDay(plot=False))
        self.indicators.append(bt.indicators.DownDayBool(plot=False))
        self.indicators.append(bt.indicators.DownMove(plot=False))
        self.indicators.append(bt.indicators.Envelope(plot=False))
        self.indicators.append(bt.indicators.ExponentialMovingAverage(plot=False))
        self.indicators.append(bt.indicators.ExponentialMovingAverageEnvelope(plot=False))
        self.indicators.append(bt.indicators.ExponentialMovingAverageOscillator(plot=False))
        self.indicators.append(bt.indicators.ExponentialSmoothing(plot=False))
        # self.indicators.append(bt.indicators.ExponentialSmoothingDynamic(plot=False))
        self.indicators.append(bt.indicators.FibonacciPivotPoint(plot=False))
        # self.indicators.append(bt.indicators.FindFirstIndex(plot=False))
        # self.indicators.append(bt.indicators.FindFirstIndexHighest(plot=False))
        # self.indicators.append(bt.indicators.FindFirstIndexLowest(plot=False))
        # self.indicators.append(bt.indicators.FindLastIndex(plot=False))
        # self.indicators.append(bt.indicators.FindLastIndexHighest(plot=False))
        # self.indicators.append(bt.indicators.FindLastIndexLowest(plot=False))
        # self.indicators.append(bt.indicators.Fractal(plot=False))
        self.indicators.append(bt.indicators.HeikinAshi(plot=False))
        self.indicators.append(bt.indicators.Highest(plot=False))
        self.indicators.append(bt.indicators.HullMovingAverage(plot=False))
        self.indicators.append(bt.indicators.HullMovingAverageEnvelope(plot=False))
        self.indicators.append(bt.indicators.HullMovingAverageOscillator(plot=False))
        self.indicators.append(bt.indicators.HurstExponent(plot=False))
        self.indicators.append(bt.indicators.Ichimoku(plot=False))
        self.indicators.append(bt.indicators.KnowSureThing(plot=False))
        self.indicators.append(bt.indicators.LaguerreFilter(plot=False))
        self.indicators.append(bt.indicators.LaguerreRSI(plot=False))
        # self.indicators.append(bt.indicators.LinePlotterIndicator(plot=False))
        self.indicators.append(bt.indicators.Lowest(plot=False))
        self.indicators.append(bt.indicators.MACD(plot=False))
        self.indicators.append(bt.indicators.MACDHisto(plot=False))
        self.indicators.append(bt.indicators.MeanDeviation(plot=False))
        self.indicators.append(bt.indicators.MinusDirectionalIndicator(plot=False))
        self.indicators.append(bt.indicators.Momentum(plot=False))
        self.indicators.append(bt.indicators.MomentumOscillator(plot=False))
        # self.indicators.append(bt.indicators.MovingAverageBase(plot=False))
        self.indicators.append(bt.indicators.MovingAverageSimple(plot=False))
        self.indicators.append(bt.indicators.MovingAverageSimpleEnvelope(plot=False))
        self.indicators.append(bt.indicators.MovingAverageSimpleOscillator(plot=False))
        # self.indicators.append(bt.indicators.NonZeroDifference(plot=False))
        # self.indicators.append(bt.indicators.OLS_BetaN(plot=False))
        # self.indicators.append(bt.indicators.OLS_Slope_InterceptN(plot=False))
        # self.indicators.append(bt.indicators.OLS_TransformationN(plot=False))
        # self.indicators.append(bt.indicators.OperationN(plot=False))
        # self.indicators.append(bt.indicators.Oscillator(plot=False))
        # self.indicators.append(bt.indicators.OscillatorMixIn(plot=False))
        self.indicators.append(bt.indicators.ParabolicSAR(plot=False))
        self.indicators.append(bt.indicators.PercentChange(plot=False))
        self.indicators.append(bt.indicators.PercentRank(plot=False))
        self.indicators.append(bt.indicators.PercentagePriceOscillator(plot=False))
        self.indicators.append(bt.indicators.PercentagePriceOscillatorShort(plot=False))
        # self.indicators.append(bt.indicators.PeriodN(plot=False))
        self.indicators.append(bt.indicators.PivotPoint(plot=False))
        self.indicators.append(bt.indicators.PlusDirectionalIndicator(plot=False))
        self.indicators.append(bt.indicators.PrettyGoodOscillator(plot=False))
        self.indicators.append(bt.indicators.PriceOscillator(plot=False))
        self.indicators.append(bt.indicators.RSI_EMA(plot=False))
        self.indicators.append(bt.indicators.RSI_SMA(plot=False))
        self.indicators.append(bt.indicators.RSI_Safe(plot=False))
        self.indicators.append(bt.indicators.RateOfChange(plot=False))
        self.indicators.append(bt.indicators.RateOfChange100(plot=False))
        # self.indicators.append(bt.indicators.ReduceN(plot=False))
        self.indicators.append(bt.indicators.RelativeMomentumIndex(plot=False))
        self.indicators.append(bt.indicators.RelativeStrengthIndex(plot=False))
        # self.indicators.append(bt.indicators.Signal(plot=False))
        self.indicators.append(bt.indicators.SmoothedMovingAverage(plot=False))
        self.indicators.append(bt.indicators.SmoothedMovingAverageEnvelope(plot=False))
        self.indicators.append(bt.indicators.SmoothedMovingAverageOscillator(plot=False))
        self.indicators.append(bt.indicators.StandardDeviation(plot=False))
        self.indicators.append(bt.indicators.Stochastic(plot=False))
        self.indicators.append(bt.indicators.StochasticFast(plot=False))
        self.indicators.append(bt.indicators.StochasticFull(plot=False))
        # self.indicators.append(bt.indicators.SumN(plot=False))
        # self.indicators.append(bt.indicators.TripleExponentialMovingAverage(plot=False))
        # self.indicators.append(bt.indicators.TripleExponentialMovingAverageEnvelope(plot=False))
        # self.indicators.append(bt.indicators.TripleExponentialMovingAverageOscillator(plot=False))
        self.indicators.append(bt.indicators.Trix(plot=False))
        self.indicators.append(bt.indicators.TrixSignal(plot=False))
        self.indicators.append(bt.indicators.TrueHigh(plot=False))
        self.indicators.append(bt.indicators.TrueLow(plot=False))
        self.indicators.append(bt.indicators.TrueRange(plot=False))
        self.indicators.append(bt.indicators.TrueStrengthIndicator(plot=False))
        self.indicators.append(bt.indicators.UltimateOscillator(plot=False))
        self.indicators.append(bt.indicators.UpDay(plot=False))
        self.indicators.append(bt.indicators.UpDayBool(plot=False))
        self.indicators.append(bt.indicators.UpMove(plot=False))
        self.indicators.append(bt.indicators.Vortex(plot=False))
        self.indicators.append(bt.indicators.WeightedAverage(plot=False))
        self.indicators.append(bt.indicators.WeightedMovingAverage(plot=False))
        self.indicators.append(bt.indicators.WeightedMovingAverageEnvelope(plot=False))
        self.indicators.append(bt.indicators.WeightedMovingAverageOscillator(plot=False))
        self.indicators.append(bt.indicators.WilliamsAD(plot=False))
        self.indicators.append(bt.indicators.WilliamsR(plot=False))
        self.indicators.append(bt.indicators.ZeroLagExponentialMovingAverage(plot=False))
        self.indicators.append(bt.indicators.ZeroLagExponentialMovingAverageEnvelope(plot=False))
        self.indicators.append(bt.indicators.ZeroLagExponentialMovingAverageOscillator(plot=False))
        self.indicators.append(bt.indicators.ZeroLagIndicator(plot=False))
        self.indicators.append(bt.indicators.ZeroLagIndicatorEnvelope(plot=False))
        self.indicators.append(bt.indicators.ZeroLagIndicatorOscillator(plot=False))
        self.indicators.append(bt.indicators.haDelta(plot=False))
        
    def trainModel(self):
        start = time.time() 
        data, target = util.prepareDataForTraining(self.datas[0], TRAINING_AMOUNT, THRESHOLD, self.indicators)
        # first model train
        self.clf.fit(data, target)
        end = time.time()
        if(DEBUG_VERBOSE):
            print(
                'Training until:', datetime.date.fromordinal(int(self.datas[0].datetime[0])),
                'Training Time:', "%.2f" % (end-start)
            )
        self.toMean['trainScores']['data'].append(self.clf.score(data, target))
        
    def predict(self):
        target = util.generateTarget(self.datas[0], TRAINING_AMOUNT, THRESHOLD)
        data = util.getDataFromIndicatorsForPrediction(TRAINING_AMOUNT, self.indicators)
        data = [data[-1]]
        target = [target[-1]]

        # data = [util.getDataFromIndicatorsForPrediction(TRAINING_AMOUNT, self.indicators)[0]]
        prediction = self.clf.predict(data)[0]
        self.toMean['predictions'].append([prediction, target[0]])

        return prediction

    def nextstart(self):
        self.trainModel()

    def next(self):
        if(len(self) % TRAINING_EXPIRE_IN) == 0:
            self.trainModel()

        prediction = self.predict()

        positionSize = self.broker.getposition(self.data).size
        if(prediction == DOWN and not positionSize < 0):
            self.sell(size=None)
        elif(prediction == UP and not positionSize > 0):
            self.buy(size=None)

    def stop(self):
        for label, measurements in self.toMean.items():
            if 'data' not in measurements:
                continue
            if(DEBUG_VERBOSE):
                print(label, np.mean(measurements['data']))
    def notify_order(self, order):
        if(order.status in [order.Canceled, order.Margin, order.Rejected]):
            print(order)
            print(self.broker.getposition(data))


In [4]:
class MySizer(bt.Sizer):
    def _getsizing(self, comminfo, cash, data, isbuy):
        sizeNecessaryToRevert = abs(self.broker.getposition(data).size)
        tradingSize = self.getSizeOnStandardDeviation(self.strategy.StandardDeviation[0])
        return tradingSize + sizeNecessaryToRevert
        
    def getSizeOnStandardDeviation(self, stddev):
        valueToTrade = RISK_TO_TRADE * self.broker.get_value()
        return math.floor(valueToTrade/stddev)

# Backtesting

In [5]:
utils.runBackTest(TESTS, NUMBER_OF_RUNS, TestStrategy, MySizer, DEFAULT_DT_FORMAT, DEFAULT_FROM_DATE)

############################
starting logistic_(20, 20, 20)
############################
end
AutoOrderedDict([('total', AutoOrderedDict([('total', 115), ('open', 1), ('closed', 114)])), ('streak', AutoOrderedDict([('won', AutoOrderedDict([('current', 0), ('longest', 8)])), ('lost', AutoOrderedDict([('current', 3), ('longest', 3)]))])), ('pnl', AutoOrderedDict([('gross', AutoOrderedDict([('total', -57.979810000011824), ('average', -0.5085948245615072)])), ('net', AutoOrderedDict([('total', -57.979810000011824), ('average', -0.5085948245615072)]))])), ('won', AutoOrderedDict([('total', 78), ('pnl', AutoOrderedDict([('total', 6168.5173499999855), ('average', 79.08355576923059), ('max', 395.234189999997)]))])), ('lost', AutoOrderedDict([('total', 36), ('pnl', AutoOrderedDict([('total', -6226.497159999997), ('average', -172.95825444444438), ('max', -846.7760400000028)]))])), ('long', AutoOrderedDict([('total', 57), ('pnl', AutoOrderedDict([('total', 396.39923999999166), ('average', 6.954372

In [6]:
for test in TESTS:
    # Plot the result
    figures = test['resultCerebro'].plot(
        style='bar',
        iplot=False,
        start=PLOT_FROM_DATE,
        # (year, month, day, hour=0, minute=0, second=0, microsecond=0, tzinfo=None, *, fold=0)
    )

    print(figures)

    figures[0][0].savefig(IMAGE_PATH + test["key"] + IMAGE_SUFIX )


[[<Figure size 1920x983 with 4 Axes>]]
[[<Figure size 1920x983 with 4 Axes>]]
[[<Figure size 1920x983 with 4 Axes>]]
[[<Figure size 1920x983 with 4 Axes>]]
[[<Figure size 1920x983 with 4 Axes>]]
[[<Figure size 1920x983 with 4 Axes>]]
[[<Figure size 1920x983 with 4 Axes>]]
[[<Figure size 1920x983 with 4 Axes>]]
[[<Figure size 1920x983 with 4 Axes>]]
[[<Figure size 1920x983 with 4 Axes>]]
